# --- Day 21: Keypad Conundrum ---

In [5]:
# --- Part One and Part Two ---

from typing import List, Dict, Tuple
from collections import deque
from itertools import product
from functools import cache

filename = "input.txt"
# filename = "test.txt" # decomment to test

NUM_PAD = {
    (0, 0): "7",
    (1, 0): "8",
    (2, 0): "9",
    (0, 1): "4",
    (1, 1): "5",
    (2, 1): "6",
    (0, 2): "1",
    (1, 2): "2",
    (2, 2): "3",
    (1, 3): "0",
    (2, 3): "A"
}

DIR_PAD = {
    (1,0): "^",
    (2,0): "A",
    (0,1): "<",
    (1,1): "v",
    (2,1): ">",
}

def find_path(keypad, start, end):
    """
    Find the shortest path from start to end by trying different path and keeping
    the one with less steps. Use BSF method
    """
    if start == end:
        return ['A'] # same posizione, just Activate
    queue = deque([(*start, "")])
    min_steps = 100 # start with an higher number
    best_path = []
    while(queue):
        x, y, path = queue.popleft()
        for nx, ny , move in [(x+1, y, '>'), (x-1, y, "<"), (x, y+1, "v"), (x,y-1,"^")]:
            if (nx, ny) not in keypad: continue
            if (nx, ny) == end and len(path) +1 > min_steps: return best_path
            if (nx, ny) == end:
                best_path.append(path + move + 'A')
                min_steps = min(len(path) + 2, min_steps)
            else:
                queue.append((nx, ny, path + move))
    return best_path

def key_sequence(keypad_paths, code):
    """
    Return a list of different movememts path needed to write the code using keypad. Path start always from 'A'.
    """
    seqs = [keypad_paths[(s, e)] for s, e in zip("A" + code, code)]
    return [''.join(segment) for segment in product(*seqs)]

# store best paths for all keys combination for each keypad
num_pad_paths = {(NUM_PAD[s], NUM_PAD[e]): find_path(NUM_PAD, s, e) for s in NUM_PAD for e in NUM_PAD}
dir_pad_paths = {(DIR_PAD[s], DIR_PAD[e]): find_path(DIR_PAD, s, e) for s in DIR_PAD for e in DIR_PAD}

@cache
def get_length(code, depth):
    if depth == 0:
        return len(code)
    length = 0
    for p1, p2 in zip("A" + code, code):
        length += min(get_length(seq, depth -1) for seq in dir_pad_paths[(p1, p2)])
    return length

# Parse file

with open(filename,"r") as file:
    codes = [c.strip() for c in file]

#print (key_sequence(num_pad_paths, "029A"))

res1 = 0
for code in codes:
    best_length = float("inf")  # a bigger number (infinity)
    for path1 in key_sequence(num_pad_paths, code):
        for path2 in key_sequence(dir_pad_paths, path1):
            for path3 in key_sequence(dir_pad_paths, path2):
                best_length = min(len(path3), best_length)
    res1 += int(code[:-1]) * best_length 
print(f"The solution for part 1 is: {res1}")

res2 = 0
for code in codes:
    best_length2 = float("inf")  # a bigger number (infinity)
    for path1 in key_sequence(num_pad_paths, code):
        best_length2 = min(best_length2 , get_length(path1, 25))
    res2 += int(code[:-1]) * best_length2 

print(f"The solution for part 2 is: {res2}")


The solution for part 1 is: 184718
The solution for part 2 is: 228800606998554
